In [1]:
%pwd

'c:\\Users\\sharm\\OneDrive\\Documents\\SwasthaBot\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\sharm\\OneDrive\\Documents\\SwasthaBot'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:

#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [6]:
extracted_data=load_pdf_file(data='Data/')

In [7]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 2627


In [9]:
from langchain.embeddings import HuggingFaceEmbeddings


In [10]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [11]:
embeddings = download_hugging_face_embeddings()

C:\Users\sharm\AppData\Local\Temp\ipykernel_7656\1196424635.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [12]:
query_result = embeddings.embed_query("Good morning")
print("Length", len(query_result))

Length 384


In [14]:
#query_result

In [13]:
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
COHERE_API_KEY = os.environ.get('COHERE_API_KEY')
if not PINECONE_API_KEY or not COHERE_API_KEY:
    raise ValueError("Set your API keys in .env file or environment variables.")



In [15]:
import pinecone
print(pinecone.__version__)


6.0.2


In [16]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

# Initialize Pinecone with the API key
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

index_name = "swasthabot"

# Create the index
pc.create_index(
    name=index_name,
    dimension=384,  # Adjust dimension as per your data
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    )
)


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-01', 'x-cloud-trace-context': '65c6135ce66d44654ed7bed703cfd2ab', 'date': 'Fri, 18 Apr 2025 06:39:38 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [17]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["COHERE_API_KEY"] = COHERE_API_KEY

In [18]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)


In [19]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [20]:
docsearch

In [21]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [22]:
retrieved_docs = retriever.invoke("What is cold?")

In [23]:
retrieved_docs

[Document(id='a0d8615f-b31a-4f11-b9be-f36558658e84', metadata={'author': 'Connie Reavis', 'creationdate': '2003-09-15T18:04:10+00:00', 'creator': 'PScript5.dll Version 5.2', 'moddate': '2003-09-15T14:07:05-04:00', 'page': 554.0, 'page_label': '555', 'producer': 'Acrobat Distiller 5.0.5 (Windows)', 'source': 'Data\\Charaka-Samhita-Acharya-Charaka.pdf', 'title': 'Microsoft Word - charakapublish2003.2.volumeI.wrd.doc', 'total_pages': 621.0}, page_content='Likewise, when cold sprinklings are applied externally the heat being \nsuppressed goes inside and destroys K hidden within.  Thus cold is overcome with cold. \n  Ci30#321-322 \n \nFurther Examples of Opposite Actions Being Caused In Unusual Circumstances: \n Very finely pounded and thick paste of even sandal [usually cooling] causes \nburning sensation due to blocking of heat in skin. \n But in thin [more watery] and coarsely pounded paste of aguru [a normally \nheating herb] causes cold.'),
 Document(id='0caccdfb-441d-4998-b76f-8a14afe

In [24]:
from langchain_community.llms import Cohere

llm = Cohere(
    model="command-r-plus",  # Change from "command-r" to "command-r-plus"
    temperature=0.4,
    max_tokens=1500
)


C:\Users\sharm\AppData\Local\Temp\ipykernel_7656\3459831383.py:3: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import Cohere``.
  llm = Cohere(


In [27]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)




In [28]:
from langchain.chains.summarize import load_summarize_chain

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [35]:
response = rag_chain.invoke({"input": "Suggest some home remedies for headache."})
print("Answer:", response["answer"])

Answer: Some home remedies for headaches include: resting in a dark, quiet room; applying a cold compress to the head or eyes; and drinking water, as dehydration can trigger headaches.


In [32]:
response = rag_chain.invoke({"input": "Suggest some home remedies for tiredness."})
print("Answer:", response["answer"])



Answer: Some home remedies for tiredness include: improving sleep habits, addressing over-sleeping and any underlying blood disorders, and adopting a wholesome diet with the best, natural foods.
